In [39]:
###IMPORT BLOCK
import alpaca_trade_api as tradeapi
from config import *
#contains API key variables KEY_ID and SECRET_KEY, as well as various URLS

import talib as ta
import talib.abstract as tab
#import bt.indicators
#Library for easily calculating various stock indicators

import requests, time, os, copy, random, pickle
import datetime as dt
import pandas as pd
import numpy as np
from urllib.error import HTTPError

In [2]:
###GLOBAL VARIABLES
APIP = tradeapi.REST(KEY_ID,SECRET_KEY,PAPER_URL,api_version='v2')
DTzero = dt.datetime(1996,9,4)
AntiMarketHours = dt.timedelta(hours=17,minutes=30) #number of hours the market _isn't_ open
PickleFile = 'AlpacaPF_P.pickle'

In [3]:
###BUY/SELL RULES
#Scalpy3, MY BOY
def buystrat_Scalpy3(dataframe):
    bardata = dataframe.assign(emalow=ta.EMA(dataframe['low'], timeperiod=10),
                              adx = ta.ADX(dataframe['high'],dataframe['low'],dataframe['close'], timeperiod=14),
                              cci = ta.CCI(dataframe['high'],dataframe['low'],dataframe['close'], timeperiod=14))
    stochf = ta.STOCHF(dataframe['high'], dataframe['low'], dataframe['close'], fastk_period=14)
    bardata = bardata.assign(fastk=stochf[0],fastd=stochf[1])
    if bardata['open'][-1] < bardata['emalow'][-1] and \
        bardata['adx'][-1] > 22 and \
        bardata['fastk'][-1] < 20 and \
        bardata['fastd'][-1] < 20 and \
        bardata['cci'][-1] <= -150.0:
        return True
    #tweaking adx here to control strength of the buy frequency
    else:
        return False
def sellstrat_Scalpy3(dataframe):
    bardata = dataframe.assign(emahigh=ta.EMA(dataframe['high'], timeperiod=10),
                              cci = ta.CCI(dataframe['high'],dataframe['low'],dataframe['close'], timeperiod=14))
    stochf = ta.STOCHF(dataframe['high'], dataframe['low'], dataframe['close'], fastk_period=14)
    bardata = bardata.assign(fastk=stochf[0],fastd=stochf[1])
    if bardata['open'][-1] >= bardata['emahigh'][-1] and \
        bardata['fastk'][-1] > 95 and \
        bardata['fastd'][-1] > 95  and \
        bardata['cci'][-1] >= 150.0:
        return True
    else:
        return False
        
    

#volumestrat41, the best of the volume-based scalping strategies (by flawed backtests)
def buystrat_vs41(dataframe):
    bardata = dataframe.assign(mfi=ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'],
                                 np.asarray(dataframe['volume'], dtype='float'), timeperiod=14),
                                 ultosc = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close']),
                                 minusdi = tab.MINUS_DI(dataframe, timeperiod=14),
                                 plusdi = tab.PLUS_DI(dataframe, timeperiod=14),
                                 rsi = tab.RSI(dataframe, timeperiod=14, price='close'),
                                 adx = tab.ADX(dataframe)
                                 )
    if bardata['adx'][-1] > 23 and \
        bardata['mfi'][-1] < 21 and \
        bardata['ultosc'][-1] < 40 and \
        bardata['rsi'][-1] < 35 and \
        bardata['minusdi'][-1] > bardata['plusdi'][-1]:
        return True
    else:
        return False
    
def sellstrat_vs41(dataframe):
    bardata = dataframe.assign(rsi = tab.RSI(dataframe, timeperiod=14, price='close'),
                                adx = tab.ADX(dataframe))
    if bardata['adx'][-1] > 23 and bardata['rsi'][-1] > 75:
        return True
    else:
        return False
    
#mountain climber (use a MinHold or there will be trouble!)
def buystrat_mclimber(dataframe):
    bardata = dataframe.assign(rsi = tab.RSI(dataframe, timeperiod=14, price='close'))
    if bardata['rsi'][-3] > 35 and bardata['rsi'][-1] > 45:
        return True
    else:
        return False           
def sellstrat_mclimber(bardata):
    bardata = dataframe.assign(rsi = tab.RSI(dataframe, timeperiod=14, price='close'))
    if bardata['rsi'][-1] > 60:
        return True
    else:
        return False

In [4]:
###OTHER FUNCTIONS
def liquidate(): #Most important function
    requests.delete(POSITIONS_URL,headers=HEADERS)
    time.sleep(5) #double delete to be certain no positions remain
    requests.delete(POSITIONS_URL,headers=HEADERS)

In [7]:
###PORTFOLIO CLASS
class Portfolio:
    def __init__(self):
        self.Pos = pd.DataFrame(columns = ['SYM','Qty','Side','Entry','Base','Current','UPLPC',
                                           'EntryTime','Favor'])
        self.Watch = pd.DataFrame(columns = ['SYM','Cooldown','Ban','Favor'])
        #self.nextmin = dt.datetime.now()+dt.timedelta(minutes=1)
        
    #Trick to find variable name of class instance from inside the class
    def findMyName(self):
        d = {v:k for k,v in globals().items()}
        self.myname = d[self]
        
        
    def addToWatch(self,SYMtxt):
        f = open(SYMtxt,'r')
        SYMlist = [sym.rstrip("\n").upper() for sym in f.readlines()]
        f.close()
        c = 0; d= 0
        for j,SYM in enumerate(SYMlist):
            if not SYM in self.Watch['SYM'].values:
                c+=1
                try:
                    if APIP.get_asset(SYM).tradable:
                        self.Watch = self.Watch.append({'SYM':SYM,'Cooldown':DTzero,'Ban':False,
                                                        'Favor':1},ignore_index=True)
                        d+=1
                    else:
                        print(SYM + ' not tradable')
                except:
                    print(SYM+ ' not found in Alpaca')
                
                if c%100 == 0: #rest the API
                    print('Resting API')
                    time.sleep(30)
        print('Tried {:d} new symbols, added {:d}'.format(c,d))

    def SyncPositions(self):
        aposlist = []
        for apos in APIP.list_positions():
            SYM = apos.symbol
            aposlist.append(SYM)
            Current = float(apos.current_price)
            UPLPC = float(apos.unrealized_plpc)
            if not SYM in self.Pos['SYM'].values: #add external position to internal
                Qty = int(apos.qty)
                Side = apos.side
                Entry = float(apos.avg_entry_price)
                Base = float(apos.avg_entry_price)
                EntryTime = dt.datetime.now()
                if not SYM in self.Watch['SYM'].values:
                    Favor = 1
                    self.Watch = self.Watch.append({'SYM':SYM,'Cooldown':DTzero,'Ban':False,
                                                    'Favor':1},ignore_index=True)
                else:
                    Favor = float(self.Watch.loc[self.Watch['SYM']==SYM,'Favor'])
                    
                self.Pos = self.Pos.append({'SYM':SYM,'Qty':Qty,'Side':Side,'Entry':Entry,
                                            'Base':Base,'Current':Current,'UPLPC':UPLPC,'EntryTime':EntryTime,
                                           'Favor':Favor},ignore_index=True)
            else: #if already internalized, just update what's already there
                self.Pos.loc[self.Pos['SYM']==SYM,['Current','UPLPC']] = [Current,UPLPC]
        #Delete internal if not in external
        for SYM in self.Pos['SYM'].values:
            if not SYM in aposlist:
                #df.drop(df[df.score < 50].index, inplace=True)
                self.Pos.drop(self.Pos[self.Pos.SYM == SYM].index,inplace=True)
                
        #Step up base if current price is greater than base price
        self.Pos.loc[self.Pos['Current'] > self.Pos['Base'],'Base'] =\
        self.Pos.loc[self.Pos['Current'] > self.Pos['Base'],'Current']
    
    def BuyWatch(self,BuyStrat,Npos,N=100,PrintTrades=True,Loaded=True):
        AACC = APIP.get_account()
        TargetCash = float(AACC.equity)/Npos
        WatchSample = self.Watch[(self.Watch['Ban'] == False) & 
                                 (dt.datetime.now() > self.Watch['Cooldown'])].sample(n=N,weights='Favor')
        for i,w in WatchSample.iterrows():
            if not w['SYM'] in self.Pos['SYM'].values:
                barset = APIP.get_barset(w['SYM'],'1Min',limit=32).df[w['SYM']]
                #barset must be at least length 28 for adx
                if barset.index[-1].tz_localize(None) < dt.datetime.now()+dt.timedelta(hours = 3)\
                -dt.timedelta(minutes=15):
                    print('CAUTION: {} barset lagging more than 15 minutes'.format(w['SYM']))
                if BuyStrat(barset):
                    shprice = barset.iloc[-1,-2] #most recent close
                    Nshares = int(TargetCash/shprice)
                    AACC = APIP.get_account()
                    if Nshares > 0 and Loaded and float(AACC.cash) > TargetCash:
                        order = APIP.submit_order(
                                    symbol = w['SYM'],
                                    qty = Nshares,
                                    side = 'buy',
                                    type = 'market',
                                    time_in_force='gtc')
                        if PrintTrades: 
                            print('Bought {:d} shares of {}'.format(Nshares,w['SYM']))
                    else:
                        if PrintTrades:
                            print('Would buy {}'.format(w['SYM']))
                            
    def SellPos(self,SellStrat,Floor=True,FloorPC=0.25,Log=True,SellLogCSV = 'AlpacaSellLog1.csv',PrintTrades=True,
                CoolTime = dt.timedelta(hours=0.5),ChangeFavor = True,
                MinHold = False, MaxHold = False,
                MinHoldTime = dt.timedelta(minutes=10),MaxHoldTime = dt.timedelta(hours=6)):
        if MaxHoldTime < MinHoldTime:
            print('WARNING: minimum hold time greater than maximum hold time. What are you doing?')
        if Log:
            if not SellLogCSV in os.listdir():
                csv = open(SellLogCSV,'w')
                csv.write('Sell,SellStrat,Symbol,EntryTime,SellTime,PLPC\n')
                csv.close()
            SellStratStr = SellStrat.__name__.lstrip('sellstrat')
            csv = open(SellLogCSV,'a')
        Now = dt.datetime.now()
        Sold = False #so it doesn't try to sell twice with multiple methods
        for i,p in self.Pos.iterrows():
            try:
                if Floor:
                    if not MinHold or Now-p['EntryTime'] > MinHoldTime: #Minhold overrides floor
                        if (p['Current']-p['Base'])/p['Base']*100 < -FloorPC:
                            order = APIP.submit_order(
                                symbol = p['SYM'],
                                qty = p['Qty'],
                                side = 'sell',
                                type = 'market',
                                time_in_force='gtc')
                            Sold = True
                            if ChangeFavor:
                                self.Watch.loc[self.Watch['SYM']==p['SYM'],'Favor'] += p['UPLPC']
                            self.Watch.loc[self.Watch['SYM']==p['SYM'],'Cooldown'] = Now+CoolTime
                            if PrintTrades: 
                                print('Floored {:d} shares of {} for {:.2f}%'.format(p['Qty'],p['SYM'],p['UPLPC']*100))
                            if Log:
                                csv.write('fsell,{},{},{},{},{:.3f}\n'.format(SellStratStr,p['SYM'],
                                                                           p['EntryTime'].strftime("%Y/%m/%d - %H:%M"),
                                                                           Now.strftime("%Y/%m/%d - %H:%M"),
                                                                           p['UPLPC'])) 
                if MaxHold and not Sold:
                    #calculate how much of real time wasn't market time
                    cal = APIP.get_calendar(p['EntryTime'].date().strftime("%Y-%m-%d"),Now.date().strftime("%Y-%m-%d"))
                    mdays = (Now.date() - p['EntryTime'].date()).days
                    mtime = (mdays-len(cal)+1)*dt.timedelta(days=1)+(len(cal)-1)*AntiMarketHours #missing time
                    if Now-p['EntryTime'] - mtime > MaxHoldTime:
                        order = APIP.submit_order(
                                symbol = p['SYM'],
                                qty = p['Qty'],
                                side = 'sell',
                                type = 'market',
                                time_in_force='gtc')
                        Sold = True
                        if ChangeFavor:
                                self.Watch.loc[self.Watch['SYM']==p['SYM'],'Favor'] += p['UPLPC']
                        if PrintTrades: 
                                print('Expired {:d} shares of {} for {:.2f}%'.format(p['Qty'],p['SYM'],
                                                                                     p['UPLPC']*100))
                        if Log:
                            csv.write('tsell,{},{},{},{},{:.3f}\n'.format(SellStratStr,p['SYM'],
                                                                       p['EntryTime'].strftime("%Y/%m/%d - %H:%M"),
                                                                       Now.strftime("%Y/%m/%d - %H:%M"),
                                                                       p['UPLPC']))            
                if not Sold:
                    if not MinHold or Now-p['EntryTime'] > MinHoldTime: #Minhold overrides sell rule
                        barset = APIP.get_barset(p['SYM'],'1Min',limit=32).df[p['SYM']]
                        if SellStrat(barset):
                            order = APIP.submit_order(
                                    symbol = p['SYM'],
                                    qty = p['Qty'],
                                    side = 'sell',
                                    type = 'market',
                                    time_in_force='gtc')
                            Sold = True
                            if ChangeFavor:
                                    self.Watch.loc[self.Watch['SYM']==p['SYM'],'Favor'] += p['UPLPC']
                            if PrintTrades: 
                                    print('Sold {:d} shares of {} for {:.2f}%'.format(p['Qty'],
                                                                                      p['SYM'],p['UPLPC']*100))
                            if Log:
                                csv.write('sell,{},{},{},{},{:.3f}\n'.format(SellStratStr,p['SYM'],
                                                                           p['EntryTime'].strftime("%Y/%m/%d - %H:%M"),
                                                                           Now.strftime("%Y/%m/%d - %H:%M"),
                                                                           p['UPLPC']))
            except (HTTPError, tradeapi.rest.APIError) as err:
                print('Error: Trying to sell ' + p['SYM'])
        if Log:
            csv.close()
       

In [21]:
###RUN ME TO TRADE (push start to RICH)
PF.SyncPositions()
nextminute = time.time()+60
while True:
    
    #Check the account
    AACC = APIP.get_account()
    if float(AACC.cash) < 0: #somehow we spent all our money
        worst = PF.Pos[PF.Pos.UPLPC == PF.Pos.UPLPC.min()]
        print('WARNING: Cash in account < 0, liquidating worst position '+worst.SYM)
        try:
            order = APIP.submit_order(
                                        symbol = worst.SYM,
                                        qty = worst.Qty,
                                        side = 'sell',
                                        type = 'market',
                                        time_in_force='gtc')
        except (HTTPError, tradeapi.rest.APIError) as err:
                print('Error: Trying to sell ' + worst.SYM)
        PF.SyncPositions()
        
    if AACC.trading_blocked or AACC.account_blocked:
        print('Something is wrong with the Alpaca account')
        break
    
    #Main loop
    PF.SellPos(sellstrat_Scalpy3,Floor=True,FloorPC=0.7)
    PF.BuyWatch(buystrat_Scalpy3,30,N=100)
    
    #Don't overload the API, wait at least a minute before looping, and do it before checking positions
    #to give them a time to update
    while nextminute-time.time()>0:
        time.sleep(1)
    nextminute = time.time()+60
            
    #Save your work and don't quit
    PF.SyncPositions()
    with open(PickleFile,'wb') as f:
        pickle.dump(PF,f)

    #Wait for market to open before putting in more orders
    APClock = APIP.get_clock()
    while not APClock.is_open:
        print('Waiting for market to open')
        time.sleep(30)
        APClock = APIP.get_clock()
        
       

CAUTION: WLTW barset lagging more than 15 minutes
Bought 3 shares of MCO
Bought 25 shares of PRGS
CAUTION: AZO barset lagging more than 15 minutes
CAUTION: IT barset lagging more than 15 minutes
Bought 12 shares of AEP
Bought 10 shares of EXR
Bought 11 shares of GRMN
Bought 6 shares of IPGP
Bought 3 shares of COO
Bought 5 shares of DG
CAUTION: AZO barset lagging more than 15 minutes
Bought 1 shares of REGN
CAUTION: AZO barset lagging more than 15 minutes
Bought 17 shares of TW
CAUTION: ENSG barset lagging more than 15 minutes
Bought 7 shares of LLY
CAUTION: NVR barset lagging more than 15 minutes
Bought 16 shares of EQR
Floored 3 shares of COO for -0.86%
CAUTION: TRNO barset lagging more than 15 minutes
Bought 49 shares of VIRT
CAUTION: AIZ barset lagging more than 15 minutes
CAUTION: ENSG barset lagging more than 15 minutes
Bought 12 shares of D
CAUTION: ENSG barset lagging more than 15 minutes
Bought 30 shares of DRE
CAUTION: TRNO barset lagging more than 15 minutes
Bought 16 shares 

CAUTION: JKHY barset lagging more than 15 minutes
Would buy VMC
Would buy PNR
Would buy ES
Would buy SYY
Would buy SYF
Would buy CTRE


KeyboardInterrupt: 

In [8]:
###RUN ME TO MAKE FIRST PF (or, if Portfolio Class Code was altered)
PF = Portfolio()
PF.addToWatch('IBD50_2020-06-01SYM.txt')
PF.addToWatch('S&P500_2020Q2SYM.txt')
PF.SyncPositions()
with open(PickleFile,'wb') as f:
    pickle.dump(PF,f)

Tried 50 new symbols, added 50
AGN not tradable
Resting API
Resting API
MKTK not found in Alpaca
Resting API
RTN not tradable
Resting API
UTX not tradable
Tried 479 new symbols, added 475


In [60]:
###RUN ME TO LOAD PF
with open(PickleFile,'rb') as f:
    PF = pickle.load(f)
PF.SyncPositions()
with open(PickleFile,'wb') as f:
    pickle.dump(PF,f)

In [40]:
liquidate()